In [1]:
#!pip install bert-tensorflow  #Python package for Google's BERT implementation.
#!pip uninstall tensorflow
!pip install tensorflow==2.0.0
!pip install tensorflow_hub

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
print("TF version: ", tf.__version__)
print("Hub version: ", hub.__version__)

TF version:  2.0.0
Hub version:  0.7.0


In [0]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

https://github.com/tensorflow/models/tree/master/official/nlp/bert (tensorflow 2 implementations for bert)

In [4]:
!pip install bert-for-tf2
!pip install sentencepiece

In [0]:
import bert
#from bert import run_classifier
from bert import tokenization
#from bert import optimization

In [6]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'BERT_OUTPUT'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = 'BUCKET_NAME' #@param {type:"string"}

if DO_DELETE:
  try:
    tf.io.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass


tf.io.gfile.makedirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: BERT_OUTPUT *****


#### Data

First, let's download the dataset, hosted by Stanford.
The code below, which downloads, extracts, and imports the IMDB Large Movie Review Dataset, is borrowed from this Tensorflow tutorial.
https://www.tensorflow.org/hub/tutorials/text_classification_with_tf_hub

In [0]:
from tensorflow import keras
import os
import re

def load_directory_data(directory):
  data = {}
  data['sentence'] = []
  data['sentiment'] = []
  for file_name in os.listdir(directory):
    with tf.io.gfile.GFile(os.path.join(directory, file_name), "r") as f:
      data['sentence'].append(f.read())
      data['sentiment'].append(re.match(r"\d+_(\d+).txt", file_name).group(1))
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos"))
  neg_df = load_directory_data(os.path.join(directory, "neg"))
  pos_df['polarity'] = 1
  neg_df['polarity'] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
  dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)
  
  print(os.path.dirname(dataset))

  train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
  test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))
  return train_df, test_df

In [41]:
train, test = download_and_load_datasets()

/root/.keras/datasets


#### To keep training fast, we'll take a sample of 5000 train and test examples, respectively.

In [0]:
train = train.sample(5000)
test = test.sample(5000)

In [43]:
train.head()

,sentence,sentiment,polarity
5308,"Gritty, dusty western from director Richard Br...",4,0
4270,I have to vote this 10 out of 10 in the rare c...,10,1
8468,*I mark where there are spoilers! Overall comm...,10,1
10187,Family problems abound in real life and that i...,10,1
10901,"Sitting, Typing Nothing is the latest ""what i...",7,1


In [11]:
train.columns

Index(['sentence', 'sentiment', 'polarity'], dtype='object')

#### For us, our input data is the 'sentence' column and our label is the 'polarity' column (0, 1 for negative and positive, respecitvely)

In [0]:
DATA_COLUMN = 'sentence'
LABEL_COLUMN = 'polarity'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]
num_labels = len(label_list)

#### Data Preprocessing
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create InputExample's using the constructor provided in the BERT library.

text_a is the text we want to classify, which in this case, is the Request field in our Dataframe.
text_b is used if we're training a model to understand the relationship between sentences (i.e. is text_b a translation of text_a? Is text_b an answer to the question asked by text_a?). This doesn't apply to our task, so we can leave text_b blank.
label is the label for our example, i.e. True, False

In [0]:
#copied from bert runclassifier.py
class InputExample(object):
  """A single training/test example for simple sequence classification."""
  def __init__(self, guid, text_a, text_b=None, label=None):
    self.guid = guid
    self.text_a = text_a
    self.text_b = text_b
    self.label = label

In [0]:
train_input_examples = train.apply(lambda x: InputExample(guid=None, #Globally unique ID for bookkeeping, unused in this example
                                 text_a = x[DATA_COLUMN],
                                 text_b = None,
                                 label= x[LABEL_COLUMN]), axis = 1)
test_input_examples = test.apply(lambda x: InputExample(guid=None, #Globally unique ID for bookkeeping, unused in this example
                                 text_a = x[DATA_COLUMN],
                                 text_b = None,
                                 label= x[LABEL_COLUMN]), axis = 1)

Next, we need to preprocess our data so that it matches the data BERT was trained on. For this, we'll need to do a couple of things (but don't worry--this is also included in the Python library):

1. Lowercase our text (if we're using a BERT lowercase model)
2. Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"])
3. Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
4. Map our words to indexes using a vocab file that BERT provides
5. Add special "CLS" and "SEP" tokens (see the readme)
6. Append "index" and "segment" tokens to each input (see the BERT paper)

Happily, we don't have to worry about most of these details.

To start, we'll need to load a vocabulary file and lowercasing information directly from the BERT tf hub module:

In [45]:
# This is a path to an uncased (all lowercase) version of BERT
#BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"  #this one supports only tf1 . it was not saved using saved model format

BERT_MODEL_HUB = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"  #use this for tf2.

#refer https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/1 for example usage
#tokens are stored as signatures
# signatures can be accessed using loaded_module.signatures[' '](*model_inputs)

#Its vocab_file is stored as tf.saved_model.Asset and 
#the do_lower_case flag is stored as a tf.Variable object on the SavedModel.
#They can be retrieved (using TensorFlow Hub 0.7.0 or newer) as follows:
#if using keras layer
#vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
#do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

# if using hub.load(model_url)
#  bert_module = hub.load(BERT_MODEL_HUB)
#  vocab_file = bert_module.vocab_file.asset_path.numpy()
#  do_lower_case = bert_module.do_lower_case.numpy()


def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  #with tf.Graph().as_default():
  bert_module = hub.load(BERT_MODEL_HUB)
  print(bert_module.vocab_file.asset_path.numpy())
  print(bert_module.do_lower_case.numpy())
  vocab_file = bert_module.vocab_file.asset_path.numpy()
  do_lower_case = bert_module.do_lower_case.numpy()
  return bert.bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

b'/tmp/tfhub_modules/03d6fb3ce1605ad9e5e9ed5346b2fb9623ef4d3d/assets/vocab.txt'
True


Great--we just learned that the BERT model we're using expects lowercase data (that's what stored in tokenization_info["do_lower_case"]) and we also loaded BERT's vocab file. We also created a tokenizer, which breaks words into word pieces:

In [16]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

Using our tokenizer, we'll call run_classifier.convert_examples_to_features on our InputExamples to convert them into features BERT understands.

In [0]:
def convert_examples_to_features(examples, label_list, max_seq_length,
                                 tokenizer):
  """Convert a set of `InputExample`s to a list of `InputFeatures`."""

  features = []
  for (ex_index, example) in enumerate(examples):
    if ex_index % 10000 == 0:
      tf.compat.v1.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

    feature = convert_single_example(ex_index, example, label_list,
                                     max_seq_length, tokenizer)

    features.append(feature)
  return features

def convert_single_example(ex_index, example, label_list, max_seq_length,
                           tokenizer):
  """Converts a single `InputExample` into a single `InputFeatures`."""

  if isinstance(example, PaddingInputExample):
    return InputFeatures(
        input_ids=[0] * max_seq_length,
        input_mask=[0] * max_seq_length,
        segment_ids=[0] * max_seq_length,
        label_id=0,
        is_real_example=False)

  label_map = {}
  for (i, label) in enumerate(label_list):
    label_map[label] = i

  tokens_a = tokenizer.tokenize(example.text_a)
  tokens_b = None
  if example.text_b:
    tokens_b = tokenizer.tokenize(example.text_b)

  if tokens_b:
    # Modifies `tokens_a` and `tokens_b` in place so that the total
    # length is less than the specified length.
    # Account for [CLS], [SEP], [SEP] with "- 3"
    _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
  else:
    # Account for [CLS] and [SEP] with "- 2"
    if len(tokens_a) > max_seq_length - 2:
      tokens_a = tokens_a[0:(max_seq_length - 2)]

  # The convention in BERT is:
  # (a) For sequence pairs:
  #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
  #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
  # (b) For single sequences:
  #  tokens:   [CLS] the dog is hairy . [SEP]
  #  type_ids: 0     0   0   0  0     0 0
  #
  # Where "type_ids" are used to indicate whether this is the first
  # sequence or the second sequence. The embedding vectors for `type=0` and
  # `type=1` were learned during pre-training and are added to the wordpiece
  # embedding vector (and position vector). This is not *strictly* necessary
  # since the [SEP] token unambiguously separates the sequences, but it makes
  # it easier for the model to learn the concept of sequences.
  #
  # For classification tasks, the first vector (corresponding to [CLS]) is
  # used as the "sentence vector". Note that this only makes sense because
  # the entire model is fine-tuned.
  tokens = []
  segment_ids = []
  tokens.append("[CLS]")
  segment_ids.append(0)
  for token in tokens_a:
    tokens.append(token)
    segment_ids.append(0)
  tokens.append("[SEP]")
  segment_ids.append(0)

  if tokens_b:
    for token in tokens_b:
      tokens.append(token)
      segment_ids.append(1)
    tokens.append("[SEP]")
    segment_ids.append(1)

  input_ids = tokenizer.convert_tokens_to_ids(tokens)

  # The mask has 1 for real tokens and 0 for padding tokens. Only real
  # tokens are attended to.
  input_mask = [1] * len(input_ids)

  # Zero-pad up to the sequence length.
  while len(input_ids) < max_seq_length:
    input_ids.append(0)
    input_mask.append(0)
    segment_ids.append(0)

  assert len(input_ids) == max_seq_length
  assert len(input_mask) == max_seq_length
  assert len(segment_ids) == max_seq_length

  label_id = label_map[example.label]
  if ex_index < 5:
    tf.compat.v1.logging.info("*** Example ***")
    tf.compat.v1.logging.info("guid: %s" % (example.guid))
    tf.compat.v1.logging.info("tokens: %s" % " ".join(
        [bert.bert_tokenization.printable_text(x) for x in tokens]))
    tf.compat.v1.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
    tf.compat.v1.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
    tf.compat.v1.logging.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
    tf.compat.v1.logging.info("label: %s (id = %d)" % (example.label, label_id))

  feature = InputFeatures(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids,
      label_id=label_id,
      is_real_example=True)
  return feature

class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self,
                input_ids,
                input_mask,
                segment_ids,
                label_id,
                is_real_example=True):
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.segment_ids = segment_ids
    self.label_id = label_id
    self.is_real_example = is_real_example

class PaddingInputExample(object):
  """Fake example so the num input examples is a multiple of the batch size.
  When running eval/predict on the TPU, we need to pad the number of examples
  to be a multiple of the batch size, because the TPU requires a fixed batch
  size. The alternative is to drop the last batch, which is bad because it means
  the entire output data won't be generated.
  We use this class instead of `None` because treating `None` as padding
  battches could cause silent errors.
  """

In [47]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128

# Convert our train and test features to InputFeatures that BERT understands.
train_features = convert_examples_to_features(train_input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = convert_examples_to_features(test_input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 5000


INFO:tensorflow:Writing example 0 of 5000


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


INFO:tensorflow:label: 0 (id = 0)


#### Creating a model

Now that we've prepared our data, let's focus on building a model. create_model does just this below. First, it loads the BERT tf hub module again (this time to extract the computation graph). Next, it creates a single new layer that will be trained to adapt BERT to our sentiment task (i.e. classifying whether a movie review is positive or negative). This strategy of using a mostly trained model is called fine-tuning.

In [48]:
import numpy as np
train_input_ids = []
train_input_mask = []
train_input_type_ids = []
train_labels_list = []

for feature in train_features:  
  train_input_ids.append(feature.input_ids)
  train_input_mask.append(feature.input_mask)
  train_input_type_ids.append(feature.segment_ids)
  train_labels_list.append(feature.label_id)

print(len(train_input_ids))

train_input_ids = np.array(train_input_ids, dtype='int32')
train_input_mask = np.array(train_input_mask, dtype='int32')
train_input_type_ids = np.array(train_input_type_ids, dtype='int32')
train_labels_list = np.array(train_labels_list)

5000


In [0]:
#train_input_ids[0:2]

In [0]:
test_input_ids = []
test_input_mask = []
test_input_type_ids = []
test_labels_list = []

for feature in train_features:  
  test_input_ids.append(feature.input_ids)
  test_input_mask.append(feature.input_mask)
  test_input_type_ids.append(feature.segment_ids)
  test_labels_list.append(feature.label_id)

test_input_ids = np.array(test_input_ids, dtype='int32')
test_input_mask = np.array(test_input_mask, dtype='int32')
test_input_type_ids = np.array(test_input_type_ids, dtype='int32')
test_labels_list = np.array(test_labels_list)


In [0]:
input_ids = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH, ), dtype=tf.int32, name='input_ids')
input_masks = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH, ), dtype=tf.int32, name='input_masks')
segment_ids = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH, ), dtype=tf.int32, name='segment_ids')

bert_layer = hub.KerasLayer(BERT_MODEL_HUB, trainable=True)
pooled_output, sequence_output = bert_layer([input_ids, input_masks, segment_ids])
drop_output = tf.keras.layers.Dropout(rate=0.1)(pooled_output)
dense_output = tf.keras.layers.Dense(2, name='dense_output', activation='softmax')(drop_output)

In [70]:
LEARNING_RATE = 2e-5
model = tf.keras.Model(inputs=[input_ids, input_masks, segment_ids], outputs=[dense_output])
print(model.summary())
#model.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
keras_layer_3 (KerasLayer)      [(None, 768), (None, 109482241   input_ids[0][0]                  
                                                                 input_masks[0][0]          

In [71]:
import numpy as np
len(train_input_ids)
print(np.array(train_input_ids).shape)
print(np.array(train_input_ids).dtype)


(5000, 128)
int32


In [72]:
train_labels_list.shape

(5000,)

In [73]:
#fit
BATCH_SIZE = 50
EPOCHS = 3
history = model.fit([train_input_ids, train_input_mask, train_input_type_ids], train_labels_list, epochs=EPOCHS, batch_size=BATCH_SIZE)  
#model.fit()

#model.fit({'input_ids': train_input_ids, 'input_masks': train_input_mask, 'segment_ids': train_input_type_ids},
#          {'dense_output': train_labels_list},
#          epochs=EPOCHS, batch_size=BATCH_SIZE)

Train on 5000 samples
Epoch 1/3
5000/5000 [==============================] - 661s 132ms/sample - loss: 0.4886 - sparse_categorical_accuracy: 0.7504
Epoch 2/3
5000/5000 [==============================] - 635s 127ms/sample - loss: 0.2241 - sparse_categorical_accuracy: 0.9120
Epoch 3/3
5000/5000 [==============================] - 632s 126ms/sample - loss: 0.0642 - sparse_categorical_accuracy: 0.9822


In [0]:
model.save('BertModel.h5')

In [58]:
test_input_type_ids.shape

(5000, 128)

In [75]:
results = model.evaluate([test_input_ids, test_input_mask, test_input_type_ids],test_labels_list, batch_size=BATCH_SIZE)

5000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [76]:
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.037
sparse_categorical_accuracy: 0.991


In [61]:
test_input_mask.shape

(5000, 128)

In [0]:
#pre_save_preds = model.predict([test_input_ids, test_input_mask, test_input_type_ids])  

#model.predict({'input_ids': test_input_ids, 'input_masks': test_input_mask, 'segment_ids': test_input_type_ids})

In [36]:
len(pre_save_preds)

1000

In [0]:
def getPrediction(in_sentences):
  labels = ["Negative", "Positive"]
  input_examples = [InputExample(guid="", text_a = x, text_b = None, label=0) for x in in_sentences]
  print(label_list)
  input_features = convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

  pred_input_ids = []
  pred_input_mask = []
  pred_input_type_ids = []
  
  for feature in input_features:  
    pred_input_ids.append(feature.input_ids)
    pred_input_mask.append(feature.input_mask)
    pred_input_type_ids.append(feature.segment_ids)
    
  pred_input_ids = np.array(pred_input_ids, dtype='int32')
  pred_input_mask = np.array(pred_input_mask, dtype='int32')
  pred_input_type_ids = np.array(pred_input_type_ids, dtype='int32')

  print(pred_input_ids.shape)
  #print(model.summary())
  
  predictions = model.predict([pred_input_ids, pred_input_mask, pred_input_type_ids])
  return predictions

In [0]:
pred_sentences = [
  "That movie was absolutely awful",
  "The acting was a bit lacking",
  "The film was creative and surprising",
  "Absolutely fantastic!"
]

In [79]:
predictions = getPrediction(pred_sentences)

[0, 1]
INFO:tensorflow:Writing example 0 of 4


INFO:tensorflow:Writing example 0 of 4


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


INFO:tensorflow:label: 0 (id = 0)


(4, 128)


In [80]:
predictions

array([[0.9959984 , 0.00400165],
       [0.8497251 , 0.15027489],
       [0.04698265, 0.9530173 ],
       [0.0581346 , 0.9418654 ]], dtype=float32)